In [4]:
'''
Docstring for models.ipynb
using spectral_feature_data.csv to train ML models and check their performance
the training and testing split is 80% - 20%
training is done using cross validation with 5 folds
models going to be used:
- Random Forest regressor
- XGBoost regressor
- CNN

'''
%pip install tensorflow scikit-learn pandas numpy tensorflow xgboost
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, r2_score
from tensorflow.keras import models, layers, Input
from tqdm import tqdm  # Progress bar
import xgboost as xgb

raw_features = pd.read_csv('spectral_feature_data.csv')
raw_features.drop(["p3.Fe.mg_kg", "p2.Zn.mg_kg"], axis=1, inplace=True)

In [5]:
#DEFINING EASY, MEDIUM, HARD FEATURES BASED ON OCCURANCE
total_rows = raw_features.shape[0]
df = pd.DataFrame()
print(raw_features.shape)
df_p_cols = [col for col in raw_features.columns if col.startswith('p')]
easy_features = []
print("Total rows in most occurance columns (>40k):")
for col in df_p_cols:
    if raw_features[col].isna().sum() < 10000:
        print(col,"\t\t", total_rows - raw_features[col].isna().sum())
        easy_features.append(col)

print("\n")
medium_features = [col for col in df_p_cols if col not in easy_features and raw_features[col].notna().sum() > 20000]
print("Total Rows in medium occurance columns (20k-40k):")
print(raw_features[medium_features].notna().sum())
hard_features = [col for col in df_p_cols if col not in easy_features and col not in medium_features]
print("\n")
print("Total Rows in hard occurance columns (<20k):") 
print(raw_features[hard_features].notna().sum())
spectral_cols = [col for col in raw_features.columns if col[0] != 'p']
print(f"\nSpectral Wavelengths: (No. of readings: {len(spectral_cols)})\n\n{spectral_cols}\n")

(44736, 34)
Total rows in most occurance columns (>40k):
p1.pH.index 		 44590
p2.N.wt_pct 		 40931
p2.OC.wt_pct 		 44624
p3.K.mg_kg 		 44488
p3.P.mg_kg 		 40764


Total Rows in medium occurance columns (20k-40k):
p1.EC.ds_m         21833
p1.Sand.wt_pct     27139
p4.CEC.cmolc_kg    22685
p4.CF.wt_pct       23237
dtype: int64


Total Rows in hard occurance columns (<20k):
p1.Clay.wt_pct          3915
p1.Silt.wt_pct          3901
p3.S.wt_pct              167
p4.BD.g_cm3             1162
p4.WR_10kPa.wt_pct       924
p4.WR_1500kPa.wt_pct     967
p4.WR_33kPa.wt_pct       919
dtype: int64

Spectral Wavelengths: (No. of readings: 18)

['410', '435', '460', '485', '510', '535', '560', '585', '610', '645', '680', '705', '730', '760', '810', '860', '900', '940']



In [6]:
print(medium_features)

['p1.EC.ds_m', 'p1.Sand.wt_pct', 'p4.CEC.cmolc_kg', 'p4.CF.wt_pct']


In [7]:
# ROW-WISE STANDARDIZATION (Standard Normal Variate - SNV) FOR SPECTRA
# ======================================================
def apply_snv(input_data):
    """
    Standard Normal Variate (SNV):
    Subtracts the Row Mean and divides by Row Std Dev.
    """
    # axis=1 means "calculate across the row"
    row_mean = input_data.mean(axis=1)
    row_std = input_data.std(axis=1)
    
    # We use numpy broadcasting to subtract/divide
    # (data - mean) / std
    snv_data = (input_data.sub(row_mean, axis=0)).div(row_std, axis=0)
    return snv_data

# Apply only to spectral columns
df_spectra_snv = apply_snv(raw_features[spectral_cols])



In [8]:
#this is to synthesize data by adding noise to existing data points
#helps to increase dataset size for better training of models especially deep learning models like CNNs
def augment_all_data(X_s, y, X_ph=None, X_ec=None, min_samples=500):
    """
    Augments Spectral data + pH + EC with sensor-specific noise profiles.
    """
    n_samples = X_s.shape[0]
    
    # 1. Skip if we have enough data
    if n_samples >= min_samples:
        return X_s, y, X_ph, X_ec
        
    # --- NOISE GENERATION ---
    
    # A. Spectral Noise (Background electronic noise)
    # 1% of the standard deviation of the spectra
    spec_noise_level = 0.01 * np.std(X_s) 
    noise_s = np.random.normal(0, spec_noise_level, X_s.shape)
    
    # B. pH Noise (Absolute Error)
    # Standard pH probe accuracy is often +/- 0.02 to 0.05
    if X_ph is not None:
        noise_ph = np.random.normal(0, 0.05, X_ph.shape) # 0.05 pH unit variance
    
    # C. EC Noise (Relative Error)
    # EC sensors usually have % error (e.g. 2%)
    if X_ec is not None:
        # Noise is 2% of the actual reading value
        noise_ec = X_ec * np.random.normal(0, 0.02, X_ec.shape)

    # --- APPLY & CONCATENATE ---
    
    # Create Noisy Copies
    X_s_new  = X_s + noise_s
    X_ph_new = (X_ph + noise_ph) if X_ph is not None else None
    X_ec_new = (X_ec + noise_ec) if X_ec is not None else None
    
    # Concatenate (Original + Noisy)
    X_s_aug = np.concatenate([X_s, X_s_new], axis=0)
    y_aug   = np.concatenate([y, y], axis=0)
    
    X_ph_aug = np.concatenate([X_ph, X_ph_new], axis=0) if X_ph is not None else None
    X_ec_aug = np.concatenate([X_ec, X_ec_new], axis=0) if X_ec is not None else None
    
    return X_s_aug, y_aug, X_ph_aug, X_ec_aug

For training the data, we will use a cnn on the easy features directly getting a 1D vector as output.\n
For the medium and hard data we will train individual models per feature getting a 0D output. \n
We will try using xgboost, and CNNs to see which model gives the best. \n
Final output will be a vector concatenating all of the outputs 

In [9]:

def build_flexible_model(input_shape, use_ph=False, use_ec=False):
    """
    Builds a model with optional pH and EC inputs.
    """
    inputs_list = []
    features_to_concat = []

    # --- 1. Spectral Branch (Always Present) ---
    input_spec = Input(shape=input_shape, name='spectral_in')
    inputs_list.append(input_spec)
    
    # Feature extraction logic
    x = layers.Conv1D(5, 3, activation='relu')(input_spec)
    #x = layers.Conv1D(5, 3, activation='relu')(x)
    x = layers.GlobalAveragePooling1D()(x)
    '''
    2nd cnn:
    x = layers.Conv1D(16, 3, activation='relu')(input_spec)
    x = layers.Conv1D(8, 3, activation='relu')(x)
    x = layers.GlobalAveragePooling1D()(x)'''
    features_to_concat.append(x)

    # --- 2. Conditional pH Branch ---
    if use_ph:
        input_ph = Input(shape=(1,), name='ph_in')
        inputs_list.append(input_ph)
        features_to_concat.append(input_ph)

    # --- 3. Conditional EC Branch ---
    if use_ec:
        input_ec = Input(shape=(1,), name='ec_in')
        inputs_list.append(input_ec)
        features_to_concat.append(input_ec)

    # --- 4. Merge & Output ---
    # If we have more than one feature source, we concatenate
    if len(features_to_concat) > 1:
        x = layers.Concatenate()(features_to_concat)
    else:
        x = features_to_concat[0] # Just the spectral features

    # Regression Head
    ''' 2nd cnn
    x = layers.Dense(16, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(8, activation='relu')(x)
    output = layers.Dense(1, name='output')(x)'''
    
    #x = layers.Dense(5, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    #x = layers.Dense(8, activation='relu')(x)
    output = layers.Dense(1, name='output')(x)

    model = models.Model(inputs=inputs_list, outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

In [10]:
#Defining dataset, Scaling the values
# dataset contains: 
# 1. scaled spectral columns
# 2. ph and ec columns
# 3. other target features

features = easy_features + medium_features + hard_features
df_final = pd.concat([df_spectra_snv, raw_features[features]], axis=1) #has snv spectral with non scaled other features
print(df_final.shape)

#these are the scaled versions of ph and ec to be used as inputs when required
scaler = StandardScaler()
ec_scaled = scaler.fit_transform(raw_features[["p1.EC.ds_m"]])
ph_scaled = scaler.fit_transform(raw_features[["p1.pH.index"]])

feature_vals = {}
features.remove("p1.pH.index")
features = [f for f in features if not f.startswith("p4")]
print(features)

for feature in features:
    feature_vals[feature] = df_final[feature].values
print(feature_vals)
print(ec_scaled, ph_scaled)






(44736, 34)
['p2.N.wt_pct', 'p2.OC.wt_pct', 'p3.K.mg_kg', 'p3.P.mg_kg', 'p1.EC.ds_m', 'p1.Sand.wt_pct', 'p1.Clay.wt_pct', 'p1.Silt.wt_pct', 'p3.S.wt_pct']
{'p2.N.wt_pct': array([ nan,  nan,  nan, ..., 0.09, 0.06, 0.22]), 'p2.OC.wt_pct': array([0.95, 0.42, 0.38, ..., 0.67, 0.57, 1.81]), 'p3.K.mg_kg': array([  78.26      ,    0.        ,    0.        , ...,  958.98424472,
       1022.83646222, 1117.31372449]), 'p3.P.mg_kg': array([ nan,  nan,  nan, ..., 29.5,  0. ,  0. ]), 'p1.EC.ds_m': array([   nan,    nan,    nan, ..., 0.1509, 0.155 , 0.1517]), 'p1.Sand.wt_pct': array([69. , 64.6, 61.3, ..., 33. , 45. , 30. ]), 'p1.Clay.wt_pct': array([15. , 20.9, 25.9, ...,  nan,  nan,  nan]), 'p1.Silt.wt_pct': array([15.9, 14.5, 12.9, ...,  nan,  nan,  nan]), 'p3.S.wt_pct': array([nan, nan, nan, ..., nan, nan, nan])}
[[        nan]
 [        nan]
 [        nan]
 ...
 [-0.33071381]
 [-0.31843349]
 [-0.32831765]] [[-1.04020046]
 [-0.89525692]
 [-0.75031338]
 ...
 [ 1.17018856]
 [ 1.29701416]
 [ 1.3441

In [11]:


def run_multi_feature_experiment(data_dictionary, X_spectral, X_ph, X_ec):
    """
    data_dictionary: {'Nitrogen': y_nitrogen_array, 'Phosphorus': y_phosphorus_array, ...}
    """
    
    # 1. Setup Storage
    master_results = []
    
    # 2. Define Configs
    configs = [
        {"name": "Spectral + pH",      "ph": True,  "ec": False},
        {"name": "Spectral + EC",      "ph": False, "ec": True},
        {"name": "Spectral + pH + EC", "ph": True,  "ec": True}
    ]
    
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    X_ph_temp = X_ph.copy() # Make a copy so we don't change original data
    X_ph_temp[X_ph_temp == 0] = np.nan

    # If X_ec has 0s that act as NaNs:
    X_ec_temp = X_ec.copy()
    X_ec_temp[X_ec_temp == 0] = np.nan

    
    # 3. Outer Loop: Iterate through Features (Nitrogen, Potassium, etc.)
    # tqdm makes a nice progress bar
    for feature_name, y_full in tqdm(data_dictionary.items(), desc="Processing Features"):
        
        print(f"\n--- Starting Feature: {feature_name} ---")
        # --- 2. Create Intelligent Mask for NaN Removal ---
        # We keep a row ONLY if: Target is valid AND pH is valid AND EC is valid
        # This ensures all 4 configs train on the exact same dataset for fair comparison.
        
        # Check where data exists (True if valid, False if NaN)
        # Assuming inputs are 1D arrays or flattened. 
        # If X_ph/X_ec are (N,1) shape, we use .flatten() just for the check
        
        y_full[y_full == 0] = np.nan 
        
        # If X_ph has 0s that act as NaNs:

        
        # --- 2. Create Intelligent Mask (Same as before) ---
        valid_y  = ~np.isnan(y_full).flatten()
        valid_ph = ~np.isnan(X_ph_temp).flatten() # Use the temp version
        valid_ec = ~np.isnan(X_ec_temp).flatten() # Use the temp version
        

        printed = False  # To ensure we print dropped rows info only once per feature
        # 4. Middle Loop: Configurations
        for config in configs:
            
            fold_maes = []
            fold_r2s = []
            fold_mapes = []

            mask = valid_y.copy() # Good practice to copy, though not strictly required
            
            # 2. Add pH requirement ONLY if the config needs it
            if config['ph']:
                mask = mask & valid_ph
                
            # 3. Add EC requirement ONLY if the config needs it
            if config['ec']:
                mask = mask & valid_ec
        
            # Apply Mask to Create "Clean" Subsets for this Feature
            X_s_clean = X_spectral[mask]
            X_p_clean = X_ph_temp[mask]
            X_e_clean = X_ec_temp[mask]
            y_clean   = y_full[mask]

            n_removed = len(y_full) - len(y_clean)
            if n_removed > 0:
                    # Optional: Print info if data was dropped
                tqdm.write(f"  > Feature '{feature_name}': Dropped {n_removed} rows due to NaNs.")

                # If data is empty after cleaning, skip
            if len(y_clean) < 100: 
                print(f"  ! Skipping {feature_name}: Not enough data points after cleaning.")
                break
                
            # 5. Inner Loop: 5-Fold Cross Validation
            if len(X_s_clean) < 10:
                continue

            for train_idx, val_idx in kf.split(X_s_clean):
                
                # --- STEP 1: EXTRACT RAW DATA ---
                x_train_s, x_val_s = X_s_clean[train_idx], X_s_clean[val_idx]
                y_train, y_val     = y_clean[train_idx],   y_clean[val_idx]
                
                # Extract pH (or set to None if not used)
                if config['ph']:
                    x_train_p = X_p_clean[train_idx]
                    x_val_p   = X_p_clean[val_idx]
                else:
                    x_train_p, x_val_p = None, None

                # Extract EC (or set to None if not used)
                if config['ec']:
                    x_train_e = X_e_clean[train_idx]
                    x_val_e   = X_e_clean[val_idx]
                else:
                    x_train_e, x_val_e = None, None

                # --- STEP 2: AUGMENTATION (Modify the variables) ---
                # This updates x_train_s, y_train, etc. to include the noisy copies
                x_train_s, y_train, x_train_p, x_train_e = augment_all_data(
                    x_train_s, y_train, x_train_p, x_train_e, min_samples=500
                )

                # --- STEP 3: BUILD INPUT LISTS (Using the NEW augmented variables) ---
                
                # Training Inputs (Augmented)
                train_inputs = [x_train_s]
                if config['ph']:
                    train_inputs.append(x_train_p)
                if config['ec']:
                    train_inputs.append(x_train_e)
                    
                # Validation Inputs (Original - never augmented)
                val_inputs = [x_val_s]
                if config['ph']:
                    val_inputs.append(x_val_p)
                if config['ec']:
                    val_inputs.append(x_val_e)


                # B. Build & Train
                # (Make sure build_flexible_model is defined in your scope)
                model = build_flexible_model(
                    input_shape=(X_s_clean.shape[1], 1),
                    use_ph=config['ph'],
                    use_ec=config['ec']
                )
                
                model.fit(
                    x=train_inputs, y=y_train,
                    validation_data=(val_inputs, y_val),
                    epochs=30, batch_size=32, verbose=0
                )
                
                # C. Evaluate
                preds = model.predict(val_inputs, verbose=0).flatten()

                mape_val = np.mean(np.abs((y_val - preds) / (y_val + 1e-6))) * 100
                
                fold_maes.append(mean_absolute_error(y_val, preds))
                fold_r2s.append(r2_score(y_val, preds))
                fold_mapes.append(mape_val)
            
            # 6. Aggregate Results for this Config
            avg_mae = np.mean(fold_maes)
            avg_r2 = np.mean(fold_r2s)
            
            
            # Add to list
            master_results.append({
                "Feature": feature_name,
                "Config": config['name'],
                "MAE": avg_mae,
                "R2": avg_r2,
                "Samples_Used": len(y_clean),
                "Error_%": np.mean(fold_mapes)
            })
            
         # 7. SAVE CHECKPOINT (Crucial for long runs)
         # Saves to CSV after every feature is done, so you never lose work.
        df_temp = pd.DataFrame(master_results)
        df_temp.to_csv("experiment_results_checkpoint(reduced neurons)(1).csv", index=False)
        print(f"Saved checkpoint for {feature_name}")

    return pd.DataFrame(master_results)

# --- How to Run It ---

# 1. Create a dictionary of your targets
# y_targets = {
#     "Nitrogen": y_n,
#     "Phosphorus": y_p,
#     "Potassium": y_k,
#     # ... add all 16 features here
# }

# 2. Run
# final_df = run_multi_feature_experiment(y_targets, X_s_clean, X_ph, X_ec)
# print(final_df)



In [12]:
def get_clean_data_for_config(X_s, y_full, X_ph, X_ec, config):
    """
    1. filters rows where Target is NaN.
    2. checks config to see if pH/EC are needed, and filters for those too.
    Returns the cleaned subsets ready for splitting.
    """
    # Base Mask: Target must exist
    mask = ~np.isnan(y_full).flatten()
    
    # Optional Masks: Only apply if the config requires that sensor
    if config['ph']:
        mask = mask & ~np.isnan(X_ph).flatten()
    if config['ec']:
        mask = mask & ~np.isnan(X_ec).flatten()
        
    return X_s[mask], y_full[mask], X_ph[mask], X_ec[mask]


def format_inputs(x_s, x_p, x_e, config, model_type):
    """
    Prepares the final input shape based on the model type.
    - XGBoost: Stacks everything into one big 2D matrix (np.hstack).
    - CNN: Returns a list of separate arrays [Spectral, pH, EC].
    """
    if model_type == 'xgboost':
        # Start with Spectral
        X_final = x_s
        # Glue columns to the right side
        if config['ph']:
            X_final = np.hstack([X_final, x_p.reshape(-1, 1)])
        if config['ec']:
            X_final = np.hstack([X_final, x_e.reshape(-1, 1)])
        return X_final

    elif model_type == 'cnn':
        # Create a list of inputs
        X_list = [x_s]
        if config['ph']: X_list.append(x_p)
        if config['ec']: X_list.append(x_e)
        return X_list
    
    else:
        raise ValueError(f"Unknown model_type: {model_type}")

def train_xgboost_fold(X_train, y_train, X_val, y_val, params):
    """Specific logic to train one XGBoost model."""
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=500,
        n_jobs=-1,
        random_state=42,
        **params # Unpack dynamic params like depth, learning_rate
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )
    return model.predict(X_val)

def train_cnn_fold(X_train, y_train, X_val, y_val, params):
    """Specific logic to train one CNN model."""
    # Note: Assumes build_flexible_model is available in scope
    input_shape = (X_train[0].shape[1], 1) # (Wavelengths, 1)
    
    model = build_flexible_model(
        input_shape=input_shape,
        use_ph=params['use_ph'], 
        use_ec=params['use_ec']
    )
    
    model.fit(
        x=X_train, y=y_train,
        validation_data=(X_val, y_val),
        epochs=30, batch_size=32, verbose=0
    )
    return model.predict(X_val, verbose=0).flatten()

def process_single_fold(X_s, y, X_p, X_e, train_idx, val_idx, config, model_type, params):
    """
    Orchestrates the workflow for ONE fold: 
    Split -> Augment -> Format -> Train -> Predict.
    """
    # 1. Split Raw Data
    x_train_s, x_val_s = X_s[train_idx], X_s[val_idx]
    y_train, y_val     = y[train_idx],   y[val_idx]
    
    # Handle Optional Inputs (Extract or set to None)
    x_train_p = X_p[train_idx] if config['ph'] else None
    x_val_p   = X_p[val_idx]   if config['ph'] else None
    
    x_train_e = X_e[train_idx] if config['ec'] else None
    x_val_e   = X_e[val_idx]   if config['ec'] else None

    # 2. Augment (Training Only)
    # Assumes augment_all_data is defined
    x_train_s, y_train, x_train_p, x_train_e = augment_all_data(
        x_train_s, y_train, x_train_p, x_train_e, min_samples=500
    )

    # 3. Format Inputs (Stacking vs List)
    train_input = format_inputs(x_train_s, x_train_p, x_train_e, config, model_type)
    val_input   = format_inputs(x_val_s, x_val_p, x_val_e, config, model_type)

    # 4. Train & Predict
    if model_type == 'xgboost':
        preds = train_xgboost_fold(train_input, y_train, val_input, y_val, params)
    elif model_type == 'cnn':
        # Pass config flags to CNN params
        params['use_ph'] = config['ph']
        params['use_ec'] = config['ec']
        preds = train_cnn_fold(train_input, y_train, val_input, y_val, params)
        
    # 5. Calculate Metrics
    mae = mean_absolute_error(y_val, preds)
    r2  = r2_score(y_val, preds)
    mape = np.mean(np.abs((y_val - preds) / (y_val + 1e-6))) * 100
    
    return mae, r2, mape

In [13]:

def run_modular_experiment(target_dict, X_spectral, X_ph, X_ec, model_type, save_file, **model_params):
    """
    Main Driver Function.
    model_type: 'xgboost' or 'cnn'
    model_params: dictionary of parameters (e.g. depth=6, learning_rate=0.05)
    """
    master_results = []
    
    configs = [
        {"name": "Spectral + pH + EC", "ph": True,  "ec": True}
    ]
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    # --- Loop 1: Features ---
    for feature_name, y_full in tqdm(target_dict.items(), desc=f"Running {model_type}"):
        
        # --- Loop 2: Configurations ---
        for config in configs:
            
            # A. Prepare Data (Helper Function)
            X_s_curr, y_curr, X_p_curr, X_e_curr = get_clean_data_for_config(
                X_spectral, y_full, X_ph, X_ec, config
            )
            
            if len(y_curr) < 10: continue

            # B. Cross Validation (Inner Loop)
            fold_maes, fold_r2s, fold_mapes = [], [], []
            
            for train_idx, val_idx in kf.split(X_s_curr):
                # Call the worker helper
                mae, r2, mape = process_single_fold(
                    X_s_curr, y_curr, X_p_curr, X_e_curr, 
                    train_idx, val_idx, 
                    config, model_type, model_params
                )
                
                fold_maes.append(mae)
                fold_r2s.append(r2)
                fold_mapes.append(mape)

            # C. Aggregate & Save
            master_results.append({
                "Feature": feature_name,
                "Model": model_type,
                "Config": config['name'],
                "MAE": np.mean(fold_maes),
                "R2": np.mean(fold_r2s),
                "Error_%": np.mean(fold_mapes),
                "Samples_Used": len(y_curr)
            })
            
            # Checkpoint Save
            pd.DataFrame(master_results).to_csv(save_file, index=False)

    return pd.DataFrame(master_results)

In [14]:
def run_dynamic_xgboost_experiment(target_dict, X_spectral, X_ph, X_ec, depth, save_file):
    master_results = []
    
    configs = [
        {"name": "Spectral Only",      "ph": False, "ec": False},
        {"name": "Spectral + pH",      "ph": True,  "ec": False},
        {"name": "Spectral + EC",      "ph": False, "ec": True},
        {"name": "Spectral + pH + EC", "ph": True,  "ec": True}
    ]
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    for feature_name, y_full in tqdm(target_dict.items(), desc="XGBoost Features"):
        
        # Base Valid Mask (Target must exist)
        base_valid_y = ~np.isnan(y_full).flatten()
        
        # Temp copies for masking checks
        X_ph_temp = X_ph.copy()
        X_ec_temp = X_ec.copy()
        valid_ph = ~np.isnan(X_ph_temp).flatten()
        valid_ec = ~np.isnan(X_ec_temp).flatten()
        
        for config in configs:
            fold_maes = []
            fold_r2s  = []
            fold_mapes = []
            
            # 1. Build Dynamic Mask
            mask = base_valid_y.copy()
            if config['ph']: mask = mask & valid_ph
            if config['ec']: mask = mask & valid_ec
            
            # 2. Extract CLEAN subsets
            X_s_curr = X_spectral[mask]
            y_curr   = y_full[mask]
            X_p_curr = X_ph_temp[mask]
            X_e_curr = X_ec_temp[mask]
            
            if len(y_curr) < 10: continue

            # --- START CV LOOP ---
            for train_idx, val_idx in kf.split(X_s_curr):
                
                # A. EXTRACT Raw Components
                x_train_s, x_val_s = X_s_curr[train_idx], X_s_curr[val_idx]
                y_train, y_val     = y_curr[train_idx],   y_curr[val_idx]
                
                # Handle optional inputs
                if config['ph']:
                    x_train_p, x_val_p = X_p_curr[train_idx], X_p_curr[val_idx]
                else:
                    x_train_p, x_val_p = None, None
                    
                if config['ec']:
                    x_train_e, x_val_e = X_e_curr[train_idx], X_e_curr[val_idx]
                else:
                    x_train_e, x_val_e = None, None

                # B. AUGMENTATION (Training Data Only)
                x_train_s, y_train, x_train_p, x_train_e = augment_all_data(
                    x_train_s, y_train, x_train_p, x_train_e, min_samples=500
                )
                
                # C. STACKING (Build Matrices)
                
                # 1. Training Matrix (Augmented)
                X_train_final = x_train_s
                if config['ph']:
                    X_train_final = np.hstack([X_train_final, x_train_p.reshape(-1, 1)])
                if config['ec']:
                    X_train_final = np.hstack([X_train_final, x_train_e.reshape(-1, 1)])
                    
                # 2. Validation Matrix (Original)
                X_val_final = x_val_s
                if config['ph']:
                    X_val_final = np.hstack([X_val_final, x_val_p.reshape(-1, 1)])
                if config['ec']:
                    X_val_final = np.hstack([X_val_final, x_val_e.reshape(-1, 1)])

                # D. TRAIN
                model = xgb.XGBRegressor(
                    objective='reg:squarederror',
                    n_estimators=500, 
                    learning_rate=0.05, 
                    max_depth=depth,
                    subsample=0.8, 
                    colsample_bytree=0.8, 
                    n_jobs=-1, 
                    random_state=42,
                    early_stopping_rounds=20
                )
                
                model.fit(
                    X_train_final, y_train,
                    eval_set=[(X_val_final, y_val)],
                    verbose=False
                )
                
                # E. EVALUATE (Corrected Order)
                # 1. Predict FIRST
                preds = model.predict(X_val_final)
                
                # 2. Calculate Metrics SECOND
                mae = mean_absolute_error(y_val, preds)
                r2  = r2_score(y_val, preds)
                # Safe MAPE calculation
                mape_val = np.mean(np.abs((y_val - preds) / (y_val + 1e-6))) * 100

                fold_maes.append(mae)
                fold_r2s.append(r2)
                fold_mapes.append(mape_val)

            master_results.append({
                "Feature": feature_name,
                "Model": "XGBoost",
                "Config": config['name'],
                "MAE": np.mean(fold_maes),
                "R2": np.mean(fold_r2s),
                "Samples_Used": len(y_curr),
                "Error_%": np.mean(fold_mapes)
            })
            
            # Save Checkpoint
            pd.DataFrame(master_results).to_csv(save_file, index=False)

    return pd.DataFrame(master_results)

In [18]:
params = {"objective":'reg:squarederror',
                    "n_estimators":300, 
                    "learning_rate":0.05, 
                    "max_depth":3,
                    "subsample":1, 
                    "colsample_bytree":1, 
                    "n_jobs":-1, 
                    "random_state":42
                    }



xgboost_4_result = run_modular_experiment(
    feature_vals,
    df_spectra_snv.values,
    ph_scaled,
    ec_scaled,
        params,
    save_file="xgboost_dynamic_results(depth = 3(clay)).csv"

    )

Running {'objective': 'reg:squarederror', 'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 3, 'subsample': 1, 'colsample_bytree': 1, 'n_jobs': -1, 'random_state': 42}:   0%|          | 0/9 [00:00<?, ?it/s]


ValueError: Unknown model_type: {'objective': 'reg:squarederror', 'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 3, 'subsample': 1, 'colsample_bytree': 1, 'n_jobs': -1, 'random_state': 42}

In [ ]:
print(feature_vals)
#xgboost_6_result = run_dynamic_xgboost_experiment(feature_vals, df_spectra_snv.values, ph_scaled, ec_scaled, 6, "xgboost_dynamic_results(depth = 6).csv")

NameError: name 'feature_vals' is not defined

In [ ]:

#xgboost_8_result = run_dynamic_xgboost_experiment(feature_vals, df_spectra_snv.values, ph_scaled, ec_scaled, 8, "xgboost_dynamic_results(depth = 8).csv")

In [ ]:
#Running the experiment (reduced neurons)
#cnn_df = run_multi_feature_experiment(feature_vals, df_spectra_snv.values, ph_scaled, ec_scaled)


In [ ]:
'''
cnn_result = pd.read_csv("experiment_results_checkpoint.csv")

print(cnn_result[cnn_result["Config"] == "Spectral + pH + EC"].sort_values(by="MAE"))
print("\n\n")
print(cnn_result[cnn_result["Config"] == "Spectral + pH"].sort_values(by="MAE"))
print("\n\n")
print(cnn_result[cnn_result["Config"] == "Spectral + EC"].sort_values(by="MAE"))
print("\n\n")
print(cnn_result[cnn_result["Config"] == "Spectral Only"].sort_values(by="MAE"))'''

'\ncnn_result = pd.read_csv("experiment_results_checkpoint.csv")\n\nprint(cnn_result[cnn_result["Config"] == "Spectral + pH + EC"].sort_values(by="MAE"))\nprint("\n\n")\nprint(cnn_result[cnn_result["Config"] == "Spectral + pH"].sort_values(by="MAE"))\nprint("\n\n")\nprint(cnn_result[cnn_result["Config"] == "Spectral + EC"].sort_values(by="MAE"))\nprint("\n\n")\nprint(cnn_result[cnn_result["Config"] == "Spectral Only"].sort_values(by="MAE"))'